Ce document présente des outils de "topic modelling" pour rapprocher des tweets sémantiquement, l'objectif étant d'identifier un tweet d'alertes concernant les transports en commun d'île de France. Ce document adapte les tutoriels https://radimrehurek.com/gensim/tut3.html sur un corpus de tweets d'alertes extrait de l'étude effectuée fin 2015 par Nextérité sur "la détection de perturbations sur le réseau de transport en commun via Twitter". Le Corpus utilisé est un ensemble de 308 tweets sélectionnés dans en premier par une requête elasticsearch puis par identification manuelle. Les modèles LSI(LSA) et LDA utilisés sont ceux du module python gensim ainsi que les transformations bag-of-words (doc2bow et tf-idf).

### Création d'un corpus d'alertes à partir du fichier d'alertes sélectionnées sur la journée du 17 avril 2015

In [1]:
import logging
import pandas as pd
from codecs import open
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

from gensim import corpora, models, similarities

data_dir='/infres/ir430/bd/stif/PYTHON/PRES_9_MARS/'

In [2]:
infilename = data_dir +'Alertes.csv'
with open(infilename, 'r',encoding='utf-8') as f:
    lines = f.readlines()

df=pd.DataFrame(lines, columns=['text'])
pd.set_option('display.max_rows', 10)
pd.set_option('max_colwidth', 1000)
df

,text
0,"RT @amertxme: ma seule joie avec la rentree c'est le matin, la demi-heure avant les cours, dans le bus avec ma musique❤❤ mais le reste fuck\r\n"
1,"Infotrafic #RERD Trafic interrompu entre Stade de France et Gare du Nord, dans ce sens (incident de signalisation) &gtInfotrafic #RERD Reprise du trafic entre Stade de France et Gare du Nord\r\n"
2,Infotrafic #RERD Prevoir jusqua 40 min de suppl. de trajet sur lensemble de la ligne (incident de signalisation) &gtRER A Cergy &lt@RERD_SNCF : on perd pas les bonnes habitudes pour 2016 20mn de retard annoncees a Combs !!!!!!\r\n
3,PID HS a Nation @RERA_RATP sur voie 2. Cc @ClientsRATP Soyez attentifs aux annonces en gare pour connaitre la deserte de votre train #qml\r\n
4,Pour bien commencer lannee avec le #RERB Deja 10 minutes de retard et je ne suis qua Massy ! #qml\r\n
...,...
303,Trafic interrompu entre aulnay et mitry #rerb accident voyageur\r\n
304,Trafic ralenti sur #RERA suite un signal d'alarme Charles de Gaulle - toile @galeRERAfr @galeRERAfr\r\n
305,Une fin de semaine ternie par un incendie La Courneuve cc @RERB et part une panne ralentissant le trafic sur @RERE_SNCF #courage #weekend\r\n
306,Usagers du #RERB prenez votre mal en patience ! Le trafic est perturb sur cette ligne #KO\r\n


### Suppression des stopwords et des caractères parasites


In [3]:
from nltk.corpus import stopwords
stop_word_list = stopwords.words('french')

In [4]:
from gensim import utils
import re

def emoji_remove(word):
    try:
        # Wide UCS-4 build
        myre = re.compile(u'['
            u'\U0001F300-\U0001F64F'
            u'\U0001F680-\U0001F6FF'
            u'\u2600-\u26FF\u2700-\u27BF]+', 
            re.UNICODE)
    except re.error:
        # Narrow UCS-2 build
        myre = re.compile(u'('
            u'\ud83c[\udf00-\udfff]|'
            u'\ud83d[\udc00-\ude4f\ude80-\udeff]|'
            u'[\u2600-\u26FF\u2700-\u27BF])+', 
            re.UNICODE)
    return(myre.sub('', word))
    
print(lines[0])

texts = [ [ emoji_remove(re.sub(r'[\\".,?!:;()\[\]\{\}/]' , "", utils.to_unicode(word)) )
          for word in line.lower().split() if word not in stop_word_list]
         for line in lines]


print(texts[0])

RT @amertxme: ma seule joie avec la rentree c'est le matin, la demi-heure avant les cours, dans le bus avec ma musique❤❤ mais le reste fuck

[u'rt', u'@amertxme', u'seule', u'joie', u'rentree', u"c'est", u'matin', u'demi-heure', u'avant', u'les', u'cours', u'bus', u'musique', u'reste', u'fuck']


On peut voir que le groupe verbal "c'est" n'est pas décomposé : on utilise un tokenizer du module nltk

In [5]:
# from nltk.tokenize import WordPunctTokenizer
# tokenizer = WordPunctTokenizer()
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import RegexpTokenizer
from itertools import chain

tweet_tokenizer = TweetTokenizer()
regexp_tokenizer = RegexpTokenizer('[^\']+')
print(lines[0])

texts = [ [ word  for word in chain.from_iterable([regexp_tokenizer.tokenize(word) 
                                        for word in tweet_tokenizer.tokenize(line.lower())]) ]
          for line in lines ]

print(texts[0])

RT @amertxme: ma seule joie avec la rentree c'est le matin, la demi-heure avant les cours, dans le bus avec ma musique❤❤ mais le reste fuck

[u'rt', u'@amertxme', u':', u'ma', u'seule', u'joie', u'avec', u'la', u'rentree', u'c', u'est', u'le', u'matin', u',', u'la', u'demi-heure', u'avant', u'les', u'cours', u',', u'dans', u'le', u'bus', u'avec', u'ma', u'musique', u'\u2764', u'\u2764', u'mais', u'le', u'reste', u'fuck']


Maintenant si on rajoute les filtres sur les emoji, la ponctuation et les stopwords

In [6]:
def transform_line(line):
    tokens = [word for word in [emoji_remove(re.sub(r'[\\".,?!:;()\[\]\{\}/]' , "", word) ) 
                            for word in chain.from_iterable([regexp_tokenizer.tokenize(word) 
                                                             for word in tweet_tokenizer.tokenize(line.lower())]) 
                            if word not in stop_word_list] 
          if len(word)>0]
    return tokens

print(lines[0])

texts = [ transform_line(line) for line in lines ]


print(texts[0])


RT @amertxme: ma seule joie avec la rentree c'est le matin, la demi-heure avant les cours, dans le bus avec ma musique❤❤ mais le reste fuck

[u'rt', u'@amertxme', u'seule', u'joie', u'rentree', u'matin', u'demi-heure', u'avant', u'les', u'cours', u'bus', u'musique', u'reste', u'fuck']


### Création d'un dictionnaire

Vectorisons maintenant cet ensemble de mots en utilisant la représentation appelée bag-of-words

In [7]:
dictionary = corpora.Dictionary(texts)
dictionary.save(data_dir +'Alertes.dict')
print(dictionary)

Dictionary(1068 unique tokens: [u'mardi', u'roi', u'reprendre', u'gt', u'roulent']...)


Testons le dictionnaire sur un nouveau tweet d'alerte

In [8]:
new_doc = "RT @LIGNEL_sncf: #RERA: Axe #Poissy: Trafic normal sur #RERA suite incident de signalisation\
hauteur de #Poissy. Lgers retards  prvoir jusqu' 07h30"
new_vec = dictionary.doc2bow(transform_line(new_doc))
new_vec

[(0, 1),
 (18, 1),
 (25, 1),
 (79, 1),
 (111, 1),
 (129, 1),
 (134, 2),
 (156, 2),
 (225, 1),
 (689, 1),
 (980, 1)]

In [9]:
words_known = [(dictionary.get(key), value) for (key,value) in new_vec]
words_known

[(u'rt', 1),
 (u'trafic', 1),
 (u'incident', 1),
 (u'axe', 1),
 (u'retards', 1),
 (u'suite', 1),
 (u'#rera', 2),
 (u'#poissy', 2),
 (u'normal', 1),
 (u'jusqu', 1),
 (u'prvoir', 1)]

On sauve les données du dictionnaire en les vectorisant cette fois avec la méthode doc2bow

In [10]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize(data_dir +'Alertes.mm', corpus)

### Concepts et transformation

On va passer d'une forme vectorielle à une autre forme vectorielle :
1. de manière à découvrir les relations entre les mots et d'utiliser ces relations pour décrire sémantiquement les données
2. afin d'avoir une représentation de la donnée plus compacte pour gagner en ressources.

#### L'analyse sémantique latente
LSI (ou parfois LSA) transforme les  documents d'un espace bag-of-words ou (de préférence) TfIdf-weighted en un espace de concepts de dimension plus réduite.
https://fr.wikipedia.org/wiki/Analyse_s%C3%A9mantique_latente

In [11]:
dictionary = corpora.Dictionary.load(data_dir +'Alertes.dict')
corpus = corpora.MmCorpus(data_dir +'Alertes.mm')
tfidf = models.TfidfModel(corpus) 

Ici  nous allons transformer notre  Tf-Idf corpus par une indexation appelée Latent Semantic Indexing.
Il est choisi ici de le faire sur dix dimensions:

In [12]:
corpus_tfidf = tfidf[corpus]
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=10) 
corpus_lsi = lsi[corpus_tfidf]
lsi.print_topics(10)

[(0,
  u'0.267*"ralenti" + 0.248*"#rera" + 0.233*"#ratp" + 0.216*"poissy" + 0.214*"maisons-laffitte" + 0.211*"vers" + 0.208*"cergy" + 0.206*"trafic" + 0.194*"panne" + 0.187*"ligne"'),
 (1,
  u'-0.226*"maisons-laffitte" + -0.225*"poissy" + 0.214*"grave" + -0.210*"cergy" + 0.207*"#rerb" + 0.205*"accident" + 0.194*"@rerb" + 0.187*"voyageur" + -0.175*"#rera" + 0.164*"#villeparisis"'),
 (2,
  u'-0.336*"destination" + -0.336*"#perturbations" + -0.334*"googl3zhvmd" + -0.305*"#retard" + -0.270*"train" + -0.261*"chelles-gournay" + -0.248*"gare" + -0.226*"#rere" + -0.205*"cohi" + 0.121*"estime"'),
 (3,
  u'0.266*"mesure" + 0.266*"scurit" + 0.264*"service" + 0.260*"fin" + -0.253*"accident" + -0.244*"grave" + 0.236*"estime" + -0.225*"voyageur" + 0.219*"reprise" + 0.202*"ligne"'),
 (4,
  u'0.297*"incident" + 0.296*"lensemble" + 0.282*"regulier" + 0.278*"termine" + 0.249*"retour" + 0.207*"a" + 0.204*"ligne" + -0.197*"maisons-laffitte" + -0.186*"poissy" + 0.178*"#ratp"'),
 (5,
  u'0.329*"rer" + 0.311

In [13]:
for doc in corpus_lsi: 
    print(doc)

[(0, 0.048953521119302651), (1, 0.045757706692598328), (2, 0.014310743913128469), (3, -0.027977169661086088), (4, 0.0031851020709016678), (5, 0.070742514515787558), (6, 0.0056314552394881893), (7, -0.024811908771991926), (8, 0.027364545904444388), (9, 0.075117127376890955)]
[(0, 0.17800332352293841), (1, 0.10850500316485762), (2, -0.066251522835089199), (3, -0.03585082532934545), (4, 0.13429515444468731), (5, 0.1061754246314533), (6, -0.20669002488328747), (7, 0.16374731124648589), (8, -0.12433975984472528), (9, -0.06582912931780914)]
[(0, 0.1359551022975895), (1, -0.029505829442711264), (2, -0.021631672470965941), (3, 0.009719491294716413), (4, 0.18024747690710216), (5, 0.038338992239854632), (6, -0.085526998667563886), (7, 0.18549804439758485), (8, -0.16994720897728205), (9, -0.01757157534830505)]
[(0, 0.065395232784641405), (1, 0.024895893188584581), (2, -0.07597004238970427), (3, -0.0092238173156893467), (4, 0.029231374920724168), (5, 0.025790069932813263), (6, -0.03184287895959842

#### Allocation de Dirichlet latente
La LDA permet aussi le passage d'un espace bag-of-words à un espace de concepts de dimension plus réduite. La LDA est une extension probabilistique de la LSI. Ainsi les concepts de LDA peuvent être interprétés comme une distribution de probabilité sur l'ensemble de mots. Cette distribution est, tout comme la LSI, induite automatiquement à partir d'un corpus d'apprentissage. Les documents de test sont ensuite, comme pour la LSI, associés à un mélange de concepts déduit de cet apprentissage.
https://fr.wikipedia.org/wiki/Allocation_de_Dirichlet_latente

In [14]:
lda = models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=10)
corpus_lda = lda[corpus_tfidf]
lda.print_topics(10)

[(0,
  u'0.007*termine + 0.007*a + 0.007*regulier + 0.007*lensemble + 0.007*retour + 0.006*#retard + 0.006*train + 0.006*googl3zhvmd + 0.006*chelles-gournay + 0.006*destination'),
 (1,
  u'0.007*ligne + 0.007*#fr + 0.006*trafic + 0.006*a + 0.006*perturbe + 0.006*regulier + 0.006*#rera + 0.006*#rerb + 0.005*mitry-claye + 0.005*ensemble'),
 (2,
  u'0.010*grave + 0.009*via + 0.009*@rerb + 0.009*voyageur + 0.009*accident + 0.008*#rerb + 0.008*reprise + 0.007*estime + 0.007*trafic + 0.007*progressivement'),
 (3,
  u'0.007*rer + 0.006*cergy + 0.006*a + 0.006*panne + 0.006*ralenti + 0.006*train + 0.006*poissy + 0.005*#qml + 0.005*rt + 0.005*plus'),
 (4,
  u'0.007*a + 0.007*b + 0.006*rer + 0.006*#rera + 0.006*jusqu + 0.006*paris + 0.005*entre + 0.005*ralenti + 0.005*ligne + 0.005*rt'),
 (5,
  u'0.008*1159 + 0.007*maisons-laffitte + 0.007*vers + 0.007*matriel + 0.007*#rera + 0.006*poissy + 0.006*panne + 0.006*accident + 0.006*cergy + 0.006*grave'),
 (6,
  u'0.005*#retard + 0.005*gare + 0.005*go

### Requêtes de similarité

L'objectif est de rappocher deux documents en fonction des concepts auxquels ils sont associés.

#### Avec LSI

In [15]:
from gensim.similarities import Similarity

new_doc = "RT @LIGNEL_sncf: #RERA: Axe #Poissy: Trafic normal sur #RERA suite incident de signalisation\
hauteur de #Poissy. Lgers retards  prvoir jusqu' 07h30"
vec_bow = dictionary.doc2bow(transform_line(new_doc))
vec_lsi = lsi[vec_bow] # convert the query to LSI space
index = similarities.MatrixSimilarity(corpus_lsi,num_features=400)
#index.save(data_dir +'Alertes.index')
#index = similarities.MatrixSimilarity.load(data_dir +'Alertes_LSI.index')
sims_to_query = index[vec_lsi]
sims = sorted(enumerate(sims_to_query), key=lambda item: -item[1])
for s in (sims):
    print ("%s,%f" % (lines[s[0]],s[1]))


@RERA_RATP un RER toutes les 30minutes pour boissy et tjs aucune annonce a part trafic ralenti! Une honte ce #rera @galeRERAfr #ratp #qml
,0.921402
19:33, le trafic est ralenti sur la ligne (panne de signalisation) #RATP #RERA
,0.915900
#incendie #lacourneuve consquences : grandes perturbations sur RER D . Circulation ralentie #RATP
,0.881850
14:45, la rame stationne a Vincennes en dir. de Boissy-Saint-Leger (incident voyageur) #RATP #RERA
,0.872351
@fafafafa Probleme de signalisation. La reparation est plus compliquee que prevue #RERB #RATP
,0.856835
RT @RERA_RATP: 17:32, le trafic est ralenti sur la ligne (incident voyageur) #RATP #RERA
,0.852981
17:32, le trafic est ralenti sur la ligne (incident voyageur) #RATP #RERA
,0.838314
17:32, le trafic est ralenti sur la ligne (incident voyageur) #RATP #RERA via @RERA_RATP
,0.826971
17:32 h le trafic est ralenti sur la ligne (incident voyageur) #RATP #RERA
,0.825178
RT @GillesKLEIN: 17:32 h le trafic est ralenti sur la ligne (incident voyag

#### Avec LDA

In [16]:
vec_lda = lda[vec_bow] # convert the query to LDA space
index = similarities.MatrixSimilarity(corpus_lda,num_features=400)
#index.save(data_dir +'Alertes.index')
#index = similarities.MatrixSimilarity.load(data_dir +'Alertes_LDA.index')
sims_to_query = index[vec_lda]
sims = sorted(enumerate(sims_to_query), key=lambda item: -item[1])
for s in (sims):
    print ("%s,%f" % (lines[s[0]],s[1]))

@Europe1 trafic trs pertub sur le RER B au sud par rpercutions
,0.827536
#rerb accident de personne  Villeparisis, trafic interrompu jusqu' 10h30
,0.827257
Ile-de-France : le trafic du RER E reste trs perturb bit.ly/1Ja7S6S
,0.826852
16:27, le trafic est ralenti sur la ligne. Reprise estimee a 17:00. (colis suspect) #RATP #RERA
,0.824489
Accident sur le #RERB, Panne lectrique sur le #RERE En gros ce matin, t'es niqu de tout les cts
,0.824432
J'ai le dmon y'a plus de rer b jusqu' 22h a cause d'une accident a la courneuve
,0.824352
14:45, la rame stationne a Vincennes en dir. de Boissy-Saint-Leger (incident voyageur) #RATP #RERA
,0.824212
@RERB panne de train a la stade de france  bonne annee avec le rer b de merde.
,0.824206
RER B : trafic fortement perturb au Nord de Paris aprs l'incendie d'un entrept 24m.fr/1IRXkfY pic.twitter.com/n9n7hSEa7E
,0.823415
RT @Oceknwls: J'ai le dmon y'a plus de rer b jusqu' 22h a cause d'une accident a la courneuve
,0.823048
Aprs un accident grave de voyag